In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import csv
import pandas as pd

In [2]:
# time
t1 = time.time()
time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(t1))


'2019-05-17 09:39:30'

In [3]:
bidInfo = {}

In [4]:
csvFile = open("namelist.csv","w+")
writer = csv.writer(csvFile)
writer.writerow(('name','url'))



10

In [5]:
sourceUrl = "https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&province=%CC%EC%BD%F2&sorder=2" \
                 "&st_param=-1&auction_start_from=2018-01-01&auction_start_to=2019-05-13&spm=a213w.3064813.9001.2"  



In [6]:
# Start the driver
driver = webdriver.Chrome()
# Setup wait for later
wait = WebDriverWait(driver, 10)

driver.get(sourceUrl)

# Store the ID of the original window
original_window = driver.current_window_handle

# Check we don't have other windows open already
assert len(driver.window_handles) == 1

In [7]:
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "page-total")))
#pages = driver.find_element_by_class_name("page-total").text


<selenium.webdriver.remote.webelement.WebElement (session="a9f5f0ab110307e9e2b0fcc5e7a62cdf", element="0.8831545459493104-1")>

In [8]:
#test page 
pages = 1

In [9]:
for page in range(1,int(pages)+1):
    if (page > 1):
        driver.find_element_by_link_text(str(page)).click()
        time.sleep(1)
    
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sf-pai-item-list")))
    data = driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML')
    jsonData =  json.loads(data)
    items = jsonData["data"]
    
    itemLen = len(items)
    for item in range(itemLen):    
    #for item in range(4):        
        bidInfo["status"] = items[item]["status"]

        
        if(bidInfo["status"] == "done"):
            
            bidInfo["id"] = items[item]["id"]
            bidInfo["title"] = items[item]["title"]
            bidInfo["consultPrice"] = items[item]["consultPrice"]
            bidInfo["currentPrice"] = items[item]["currentPrice"]
            bidInfo["bidCount"] = items[item]["bidCount"]
            bidInfo["delayCount"] = items[item]["delayCount"]
            bidInfo["applyCount"] = items[item]["applyCount"]

            url = "#pai-item-"+str(bidInfo["id"])
            # Click the link which opens in a new window
            driver.find_element_by_css_selector(url).click()
            time.sleep(1) # have to wait sometime

            # Wait for the new window or tab
            wait.until(EC.number_of_windows_to_be(2))
            handles = driver.window_handles
            driver.switch_to.window(handles[1])
            #
            url2 = str(driver.current_url)+"#J_Confirmation"
            driver.get(url2)
            
            
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "content-wrap")))
            info = driver.find_element_by_class_name("c-content").text
            index = info.find("通过")
            
            # getName
            bidInfo["name"] = info[4:index]
            # getDate
            bidInfo["date"] = driver.find_element_by_class_name("c-stamp").text
            

            
            print("Name:"+bidInfo["name"]+": Page :"+str(page)+"   Item :"+str(item))
            print("url:"+driver.current_url)
            print("id:"+str(bidInfo["id"])+" title: "+bidInfo["title"])
            print("consultPrice:"+str(bidInfo["consultPrice"]) + " currentPrice:" +str(bidInfo["currentPrice"]))
            print("bidCount:"+str(bidInfo["bidCount"]) +" delayCount:"+str(bidInfo["delayCount"])+" applyCount:"+str(bidInfo["applyCount"]))
            print("date:"+bidInfo["date"])
            # add to file
            writer.writerow((bidInfo["name"],driver.current_url))
            driver.close()
            driver.switch_to.window(handles[0])
        else:
            print("***item failed******"+": Page :"+str(page)+"   Item :"+str(item))



***item failed******: Page :1   Item :0
***item failed******: Page :1   Item :1
***item failed******: Page :1   Item :2
Name:杨宁: Page :1   Item :3
url:https://sf-item.taobao.com/sf_item/591345585914.htm?spm=a213w.7398504.paiList.4.4b4c26cdhMJiXZ#J_Confirmation
id:591345585914 title: 天津市西青区泰佳道与汇川路交口东南侧武台馨苑13-1-2205号
consultPrice:2970000.0 currentPrice:2630000.0
bidCount:50 delayCount:17 applyCount:9
date:2019年05月13日
Name:魏环利: Page :1   Item :4
url:https://sf-item.taobao.com/sf_item/593056639808.htm?spm=a213w.7398504.paiList.5.4b4c26cdhMJiXZ#J_Confirmation
id:593056639808 title: 天津市和平区西安道与长沙路交口东北侧诚基经贸中心1-2-902
consultPrice:3620000.0 currentPrice:3724000.0
bidCount:64 delayCount:53 applyCount:11
date:2019年05月13日
Name:李玉芹: Page :1   Item :5
url:https://sf-item.taobao.com/sf_item/592456853691.htm?spm=a213w.7398504.paiList.6.4b4c26cdhMJiXZ#J_Confirmation
id:592456853691 title: 天津市河东区津塘路丰盈里4-303房地产
consultPrice:3320000.0 currentPrice:3079800.0
bidCount:19 delayCount:17 applyCount:3
date:2019年

In [10]:
driver.quit()

In [11]:
csvFile.close()
csv = pd.read_csv('namelist.csv', encoding='utf-8')
csv.to_excel('namelist.xlsx', sheet_name='data')

In [12]:
t2 = time.time()
time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(t2))

seconds = t2 - t1
m, s = divmod(seconds, 60)
h, m = divmod(m, 60)
print ("%02d:%02d:%02d" % (h, m, s))


00:00:28
